<a href="https://colab.research.google.com/github/Salvoaf/labComputerVision/blob/main/5_DAAT_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document-at-a-time Query Processing

Implement document-at-a-time query processing using a simple scoring function.

In [65]:
!pip install ipytest

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [66]:
import ipytest
import pytest

from typing import Dict, List, Tuple
from collections import Counter

ipytest.autoconfig()

### Inverted index

For simplicity, the inverted index for the document collection is given as a dictionary, with a terms as keys and posting lists as values. Each posting is a (document ID, term frequency) tuple.

In [80]:
"beijing duck recipe"
index = {
    "beijing": [        (1, 1),                 (4, 1)],
    "dish":    [        (1, 1),                 (4, 1)],
    "duck":    [(0, 3), (1, 2), (2, 2),         (4, 1)],
    "rabbit":  [                (2, 1), (3, 1)        ],
    "recipe":  [                (2, 1), (3, 1), (4, 1)]
}

### Document lengths

The length of each document is provided in a list (Normally, this information would be present in a document index).



In [68]:
doc_len = [3, 4, 4, 2, 4]

### Document-at-a-time scoring

The retrieval function we use is the following:

$$score(q,d) = \sum_{t \in q} w_{t,d} \times w_{t,q}$$

where $w_{t,d}$ and $w_{t,q}$ are length-normalized term frequencies, i.e., $w_{t,d} = \frac{c_{t,d}}{|d|}$, where $c_{t,d}$ is the number of occurrences of term $t$ in document $d$ and $|d|$ is the document length, i.e., the total number of terms. Similarly for the query.

We utilize the fact that the posting lists are ordered by document ID. Then, it's enough to iterate in parallel through term query term's posting list, and score the minimum docid at each iteration. We keep a pointer for each query term and we move it forward every time a docid is scored.


We use an helper function that, given a dict of posting list pointers `pos`, returns the minimum docid. If all pointers go beyond the end of the corresponding posting list, we return `num_docs`, a special docid that it is guaranteed to not appear in any posting list.

In [69]:
def min_docid(index: Dict[str, List[Tuple[int, int]]], 
              pos: Dict[str, int],
              num_docs: int, actualdocid) -> int:
    """Returns the minimum docid across posting lists.
    
    Args:
        index: Dict holding the inverted index.
        pos: the current positions in the query posting lists.
        num_docs: the number of indexed documents.
    
    Returns:
        the minimum docid across the posting lists, or num_docs if all 
        postings in all posting lists have been processed.
    """
    min_docid = num_docs
    # For each query term posting list.
    for term, pointer in index.items():
      for docid in pointer:
          if docid[0] < min_docid and docid[0] > actualdocid:
            min_docid = docid[0]  

         # TODO: check if the posting list contains a valid docid; if so, update min_docid
        
    # Return the min docid computed.
   
    return min_docid

[(1, 1), (4, 1)]
[(1, 1), (4, 1)]
[(0, 3), (1, 2), (2, 2), (4, 1)]
[(2, 1), (3, 1)]
[(2, 1), (3, 1), (4, 1)]


In [160]:
def score_collection(index: Dict[str, List[Tuple[int, int]]], 
                      doc_len: List[int], 
                      query: str) -> List[Tuple[int, float]]:
    """Scores all documents in the collection.
    
    Args:
        index: Dict holding the inverted index.
        doc_len: List with document lengths.
        query: Search query.
    
    Returns:
        List with (document_id, score) tuples, ordered by score desc.
    """
    
    # Turns the query string into a "term: freq" dictionary.
    query_freqs = dict(Counter(query.split()))

    # Computes query length (i.e., sum of all query term frequencies).
    query_len = sum(query_freqs.values())

    doc_scores = {}  # Holds the final document scores (this should be a priority list, but for simplicity we use a dictionary here).
    
    pos = {term: 0 for term in query_freqs}  # Holds a pointer for each query term's posting list.
    
    # The starting docid.
    docid = min_docid(index, pos, len(doc_len),-1)
    # While the posting lists have not been completely traversed.
    vector_docid =[]
    doc_score = []
    position = 0
    matrix_index = []
    while docid != len(doc_len):
        vector_docid.append(docid)
        score = 0.0
        scalar_list_index = []
        
        mod_d = 0
        
        for term, postlisting in index.items():
          score = 0.0
          for id, frq in postlisting:
            if id == docid:
              c = frq
              mod_d = doc_len[position]
              score = c/mod_d
          scalar_list_index.append(score)
          
        matrix_index.append(scalar_list_index)

        
        scalar_list_query = []
        for term, freq_query in index.items():
          if term in query_freqs.keys():
            cq = query_freqs[term]  #ricorrenza della parola in quella query
            mod_q = len(query_freqs) #lunghezza query
            score = cq/mod_q
            scalar_list_query.append(score)
          else:
            scalar_list_query.append(0.0)
        

        
        # Update the docid.
         
        docid = min_docid(index, pos, len(doc_len),docid)
        position = position +1
    for j, vector in enumerate(matrix_index):
      val = 0.0
      
      for i, elem in enumerate(vector):
        val = val + scalar_list_query[i]*elem
        
      doc_score.append([j,val])
    # TODO: return doc_scores sorted
    

    if doc_score != []:
      return doc_score
    return None


#### Tests

In [166]:
scores = score_collection(index, doc_len, "beijing duck recipe") 
scores

[[0, 0.3333333333333333],
 [1, 0.25],
 [2, 0.25],
 [3, 0.16666666666666666],
 [4, 0.25]]

In [167]:
%%ipytest

def test_scoring():
    scores = score_collection(index, doc_len, "beijing duck recipe ")    
    assert scores[0][0] == 0
    assert scores[0][1] == pytest.approx(1/3, rel=1e-2)
    assert scores[2][0] == 2
    assert scores[2][1] == pytest.approx(1/4, rel=1e-2)
    assert scores[3][0] == 3
    assert scores[3][1] == pytest.approx(1/6, rel=1e-2)


.                                                                                            [100%]
1 passed in 0.02s
